In [4]:
import requests
import json
import pymysql

# 请求地址
url = 'http://push2.eastmoney.com/api/qt/stock/details/get?secid=10.10002268&ut=bd1d9ddb04089700cf9c27f6f7426281&fields1=f1,f2,f3,f4&fields2=f51,f52,f53,f54,f55&pos=-11&cb=jQuery18309999433802836603_1580699529194&_=1580699529198'


# get请求
json_str = requests.get(url)
# 请求数据转为text
json_data = json_str.text

# 根据括号'('与反括号')'的索引截取
json_data = json_data[json_data.index('(')+1 : json_data.index(')')]

# 转为json
json_data = json.loads(json_data)

# 获取json的data
data_list = json_data.get('data')

# 获取data中的details对应列表
details_list = data_list['details']

# 为了给mysql插入多条数据，details_list改造成为多个元组组成的list，一个元组对应一条数据
details_list = [tuple(i.split(',')) for i in details_list]


# 给数据库一次插入多条数据
# 创建连接
con = pymysql.connect(host='localhost', user='root', password='123456@Wky', database='push2_eastmoney', port=3306)

# 创建游标
cur = con.cursor()

# sql语句
sql = '''
    insert into details(time, price, volume, f3, f4) values(%s, %s, %s, %s, %s)
'''
args = details_list
try:
    # 执行sql
    cur.executemany(sql, args) # 插入多条数据，就是多个元组组成的list
    # 执行没问题，就提交事务
    con.commit()
    print('插入数据成功！！')
except Exception as e:
    print(e)
    con.rollback()
    print('插入数据失败！')
finally:
    con.close()

插入数据成功！！
